In [ ]:
import pandas as pd
import glob
import numpy as np

In [ ]:
#ファイル読み込み
df = pd.DataFrame()
for i in glob.glob("train/*.csv*"):
    tmp_df=pd.read_csv(i)
    df = pd.concat([df, tmp_df],ignore_index=True)

In [ ]:
#欠損値確認,欠損値削除
df_rownum=df.shape[0]
df_null_column=df.columns[df.isnull().sum()==df_rownum]
df=df.drop(df_null_column,axis=1)

In [ ]:
df.drop(['ID',
         '種類',
#                '地区名',
#                '最寄駅：名称',
        '市区町村コード',
#                '用途',
#                '市区町村名',
#                '間取り',
#                '建物の構造',
#                '都市計画'
              ],inplace=True,axis=1)
df["取引の事情等"].fillna(0, inplace=True)
df.loc[df["取引の事情等"] != 0, "取引の事情等"] = 1

In [ ]:
#欠損値の多い列を削除
df=df.drop(['今後の利用目的'],axis=1)

In [ ]:
#各列のユニークな値を抽出
unique_value=df.apply(pd.Series.unique)
# print(unique_value['都道府県名'])

In [ ]:
df_edit1=df.copy()
df_edit1.dropna(inplace=True)
df_edit1.loc[df_edit1['建築年']=='戦前','建築年']='昭和20年'
df_edit1['和暦名称']=df_edit1['建築年'].str[:2]
df_edit1['西暦']=df_edit1['建築年'].str[2:-1]
df_edit1['西暦'].unique()
df_edit1['西暦']=df_edit1['西暦'].astype(int)
df_edit1.loc[df_edit1['和暦名称']=='昭和','西暦']+=1925
df_edit1.loc[df_edit1['和暦名称']=='平成','西暦']+=1988
df_edit1.loc[df_edit1['和暦名称']=='令和','西暦']+=2018
df_edit1.drop(['和暦名称','建築年'],inplace=True,axis=1)
df_edit1.loc[df_edit1['最寄駅：距離（分）']=='30分?60分','最寄駅：距離（分）']=45
df_edit1.loc[df_edit1['最寄駅：距離（分）']=='1H?1H30','最寄駅：距離（分）']=75
df_edit1.loc[df_edit1['最寄駅：距離（分）']=='1H30?2H','最寄駅：距離（分）']=105
df_edit1.loc[df_edit1['最寄駅：距離（分）']=='2H?','最寄駅：距離（分）']=120
df_edit1['取引時点年数']=df_edit1['取引時点'].str[:4]
df_edit1['四半期']=df_edit1['取引時点'].str[5:]
df_edit1.loc[df_edit1['四半期']=='第2四半期','取引時点年数']+='.25'
df_edit1.loc[df_edit1['四半期']=='第3四半期','取引時点年数']+='.5'
df_edit1.loc[df_edit1['四半期']=='第4四半期','取引時点年数']+='.75'
df_edit1.drop(['取引時点'],
#                '四半期'],
              inplace=True,axis=1)

In [ ]:
df_edit1.loc[df_edit1['面積（㎡）']=='2000㎡以上','面積（㎡）']='2000'

In [ ]:
df_edit1['最寄駅：距離（分）']=df_edit1['最寄駅：距離（分）'].astype(float)
df_edit1['面積（㎡）']=df_edit1['面積（㎡）'].astype(int)
df_edit1['西暦']=df_edit1['西暦'].astype(int)
df_edit1['取引時点年数']=df_edit1['取引時点年数'].astype(float)
df_edit1['容積率（％）']=df_edit1['容積率（％）'].astype(float)
df_edit1['建ぺい率（％）']=df_edit1['建ぺい率（％）'].astype(float)
df_edit1['取引価格（総額）_log']=df_edit1['取引価格（総額）_log'].astype(float)

In [ ]:
df_test=df_edit1['取引価格（総額）_log']
df_edit1.drop('取引価格（総額）_log',inplace=True,axis=1)

In [ ]:
# df_dummy=pd.get_dummies(df_edit1[[
# #     '市区町村コード',
#     '都道府県名',
#     '市区町村名',
#     '最寄駅：名称',
#      '間取り',
#      '建物の構造',
# #     '用途',
#     '都市計画',
#     '改装']],drop_first=True)
# df_dummy = pd.concat([df_edit1, df_dummy], axis=1)
# df_dummy.drop(['都道府県名', 
#                '市区町村名',
#                '間取り',
#                 '建物の構造',
#                 '都市計画',
#                '改装'],axis=1,inplace=True)

In [ ]:
from sklearn.preprocessing import OrdinalEncoder
oe = OrdinalEncoder()

labels = df_edit1[[
#     '市区町村コード',
    '都道府県名',
    '市区町村名',
    '最寄駅：名称',
     '間取り',
     '建物の構造',
    '地区名',
     '用途',
    '都市計画',
    '四半期',
    '改装']].values
labels_id = oe.fit_transform(labels)
# df_dummy = pd.concat([df_edit1, labels_id], axis=1)
# df_dummy.drop(['都道府県名', 
#                '市区町村名',
#                '間取り',
#                 '建物の構造',
#                 '都市計画',
#                '改装'],axis=1,inplace=True)
df_dummy = pd.concat([df_edit1.reset_index(drop=True), pd.DataFrame(labels_id)], axis=1)
df_dummy.drop(['都道府県名',
               '市区町村名', 
               '四半期',
               '間取り', 
               '建物の構造', 
               '都市計画',
               '最寄駅：名称',
               '地区名',
               '用途',
               '改装'], axis=1, inplace=True)
print(labels_id.shape) 

In [ ]:
df_dummy

In [ ]:
df_dummy['面積（㎡）']=np.log(df_dummy['面積（㎡）'])
df_dummy['面積（㎡）'].replace([np.inf, -np.inf], np.nan, inplace=True)
df_dummy['面積（㎡）'].fillna(0,inplace=True)

In [ ]:
df_dummy['最寄駅：距離（分）']=np.log(df_dummy['最寄駅：距離（分）'])
df_dummy['最寄駅：距離（分）'].replace([np.inf, -np.inf], np.nan, inplace=True)
df_dummy['最寄駅：距離（分）'].fillna(0,inplace=True)

In [ ]:
df_dummy['建ぺい率×容積率']=df_dummy['建ぺい率（％）']*df_dummy['容積率（％）']

In [ ]:
df_dummy['敷地面積']=df_dummy['面積（㎡）']/df_dummy['建ぺい率（％）']*100

In [ ]:
df_dummy['延べ床面積']=df_dummy['容積率（％）']/100*df_dummy['敷地面積']

In [ ]:
df_dummy['年差']=df_dummy['西暦']-df_dummy['取引時点年数']

In [ ]:
df_dummy.isnull().sum()

In [ ]:
# df_dummy['最寄駅：距離（分）'].max()

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_dummy, df_test, test_size=0.2, random_state=41)

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge,Lasso
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
import xgboost as xgb
import numpy as np
xgb_r = xgb.XGBRegressor(objective='reg:linear',
                          n_estimators=500,
                          max_depth=8,
#                           gamma=5,
                          seed=123)

pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('regressor',xgb_r)
])
pipe.fit(X_train, y_train)
y_pred = pipe.predict(X_train)
# y_trueは実際の値、y_predは予測値
mse = mean_squared_error(y_train, y_pred)
rmse = np.sqrt(mse)
print("RMSE:", rmse)
y_pred = pipe.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
mae=mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mse)
print("RMSE:", rmse)
print(mae)

In [ ]:
# coefficients = pipe.named_steps['regressor'].coef_
# # 各特徴量の係数を出力
# for i, col_name in enumerate(X_train.columns):
#     print(f"{col_name}: {coefficients[i]}")

In [ ]:
pipe.named_steps

In [ ]:
df_dummy

In [ ]:
import seaborn as sns
import japanize_matplotlib
import matplotlib.pyplot as plt
# sns.set(font='Hiragino Sans')zz
sns.pairplot(df_dummy[['最寄駅：距離（分）', '面積（㎡）', '建ぺい率（％）','容積率（％）','西暦','取引時点年数',]])

In [ ]:
plt.scatter(x=(df_dummy['西暦']), y=np.log(df_dummy['面積（㎡）']))

In [ ]:
df_dummy['最寄駅：距離（分）']

In [ ]:
np.log(df_dummy['最寄駅：距離（分）']).isnull().sum()

In [ ]:
import japanize_matplotlib
import matplotlib.pyplot as plt
plt.hist(df_dummy['西暦']-df_dummy['取引時点年数'])